---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [35]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [14]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [15]:
def graph_identification():
    
    # Your Code Here
    
    return ['PA','SW_L','SW_L','PA','SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [3]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [12]:
def salary_predictions():
    
    # Your Code Here
    df = pd.DataFrame(index=G.nodes())
    
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    closeCent = nx.closeness_centrality(G)
    df['closeCent'] = pd.Series(closeCent)
    btwnCentNorm_woendpoints = nx.betweenness_centrality(G,normalized=True,endpoints=False)
    df['btwnCentNorm_woendpoints'] = pd.Series(btwnCentNorm_woendpoints)
    df['clustering'] = pd.Series(nx.clustering(G))
    hits = nx.hits(G, max_iter=100, normalized=True)
    df['auth'] =  pd.Series(hits[1])

    test_df = df[df['ManagementSalary'].isnull()]
    main_df = df[~df['ManagementSalary'].isnull()]
    
    X = main_df[['closeCent','btwnCentNorm_woendpoints','clustering','auth']]
    y = main_df['ManagementSalary']

    test_df = test_df[['closeCent','btwnCentNorm_woendpoints','clustering','auth']]

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    # we must apply the scaling to the test set that we computed for the training set
    X_test_scaled = scaler.transform(X_test)
    #print('Feature importances: {}'.format(clf.feature_importances_))
    test_df_scaled = scaler.transform(test_df)

    
    
    #clf = SVC(kernel='rbf', C=1, gamma=100,probability=True)
    #y_proba = clf.fit(X_train_scaled, y_train).predict_proba(test_df_scaled)
    
    clf = RandomForestClassifier()
    y_proba = clf.fit(X_train_scaled, y_train).predict_proba(test_df_scaled)
    
    #clf = GradientBoostingClassifier(learning_rate = .01, max_depth = 2, random_state = 0)
    #y_proba = clf.fit(X_train.reshape(-1,1), y_train).predict_proba(test_df.reshape(-1,1))


    
    return pd.Series(y_proba[:,1], index=test_df.index, dtype="float64")

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [36]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [20]:
def new_connections_predictions():
    
    # Your Code Here
    
    return # Your Answer Here

In [38]:
future_connections['n1'] = [ x for (x,y) in future_connections.index]


In [39]:
future_connections['n2'] = [ y for (x,y) in future_connections.index]

In [40]:
future_connections['n1'] = future_connections['n1'].astype(int)
future_connections['n2'] = future_connections['n2'].astype(int)
#future_connections['Future Connection'] = future_connections['Future Connection'].astype(int)

In [41]:
G = nx.from_pandas_dataframe(future_connections, 'n1', 'n2', edge_attr='Future Connection')

# 

In [29]:
future_connections['PA'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]

In [32]:
future_connections[0:5]

,Future Connection,n1,n2,PA
"(6, 840)",0.0,6,840,880308
"(4, 197)",0.0,4,197,881790
"(620, 979)",0.0,620,979,1000998
"(519, 872)",0.0,519,872,974153
"(382, 423)",0.0,382,423,964035


In [46]:
#future_connections['CN'] = future_connections.index.map(lambda fc: len(list(nx.common_neighbors(G, fc[0], fc[1]))))
CN_list = list(nx.jaccard_coefficient(G,(6,840)))

TypeError: 'int' object is not iterable

[(0.0, 1.0, 0.9373104145601617),
 (0.0, 5.0, 0.8096676737160121),
 (0.0, 6.0, 0.8548548548548549),
 (0.0, 266.0, 0.9311377245508982),
 (0.0, 268.0, 0.9484848484848485),
 (0.0, 17.0, 0.8910386965376782),
 (0.0, 18.0, 0.9313131313131313),
 (0.0, 146.0, 0.9496981891348089),
 (0.0, 148.0, 0.9441117764471058),
 (0.0, 283.0, 0.8407258064516129),
 (0.0, 166.0, 0.8182741116751269),
 (0.0, 297.0, 0.9587525150905433),
 (0.0, 560.0, 0.9283551967709385),
 (0.0, 177.0, 0.9451219512195121),
 (0.0, 178.0, 0.935871743486974),
 (0.0, 309.0, 0.9532520325203252),
 (0.0, 313.0, 0.9647177419354839),
 (0.0, 316.0, 0.9482758620689655),
 (0.0, 64.0, 0.8161616161616162),
 (0.0, 65.0, 0.8864321608040201),
 (0.0, 581.0, 0.936),
 (0.0, 73.0, 0.9605263157894737),
 (0.0, 74.0, 0.9152196118488254),
 (0.0, 459.0, 0.9179331306990881),
 (0.0, 215.0, 0.9142857142857143),
 (0.0, 88.0, 0.9292123629112662),
 (0.0, 218.0, 0.9367991845056065),
 (0.0, 221.0, 0.9379450661241099),
 (0.0, 222.0, 0.9209726443768997),
 (0.0, 223.0